
# Empty ARF Fiber

What does the wavelength sensitivity look like for the ARF fiber without microstructures.  Here we look at the modes.

In [ ]:
import numpy as np

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from fiberamp.fiber.microstruct.pbg import ARF2
import matplotlib.pyplot as plt


In [ ]:
B = ARF2(name='fine_cladding', poly_core=True, refine=0,
         curve=8, shift_capillaries=False)

d = (B.T_cladding )* B.scale

n1, n2 = 1.00027717, 1.4388164768221814
lines = [2 * n1 * d / m * ((n2/n1)**2 - 1)**.5 for m in range(11, 21)]

# for l in lines:
#     plt.plot([l,l],[-1,1])

In [ ]:
t0 = B.R_cladding * B.scale
t1 = B.T_cladding * B.scale
n_air = 1.00027717
n_glass = 1.4388164768221814


In [ ]:
t0, t1

In [ ]:
A = BraggExact(ts=[t0, t1, .5*t0], mats=['air', 'glass', 'air'], 
               ns=[lambda x:n_air, lambda x:n_glass, lambda x:n_air],
              maxhs=[.2,.02, .2], wl=1.8e-6, scale=B.scale)

In [ ]:
Draw(CF([n for n in A.ns]), A.mesh)

In [ ]:
# k_low = A.k0 * A.ns[0] * A.scale
k_low, k_high = A.k0 * np.min(A.ns) * A.scale, A.k0 * np.max(A.ns) * A.scale

outer = 'h2'
Ktype = 'kappa'
nu = 1

In [ ]:
%matplotlib inline
plotlogf(A.determinant, k_low-.1, k_high+.1, -.1,.1, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
%matplotlib notebook
plotlogf(A.determinant, .999*k_low, 1.0001*k_low, -.01,.01, nu, outer,
         iref=100, rref=100, levels=100, figsize=(12,8))

In [ ]:
guess = np.array(.9999*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer, Ktype), tol = 1e-16)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer, Ktype)))


# Bragg Exact Fields

In [ ]:
Fs = A.all_fields(beta1, nu=nu, outer=outer, Ktype=Ktype)

In [ ]:
Draw(Fs['Ez'], A.mesh)

In [ ]:
Draw(Fs['Hz'], A.mesh)

In [ ]:
Draw(Fs['Etv'].real, A.mesh, vectors={'grid_size':150})


In [ ]:
Draw(Fs['Ephi'], A.mesh)

In [ ]:
Draw(Fs['Hphi'], A.mesh)

# Add Polymer layers

In [ ]:
k = 1e-3
n_soft = 1.35 - k * 1j
n_hard = 1.45 - k * 1j


In [ ]:
A = BraggExact(ts=[t0, t1, 30*t0, 30*t0, 5*t0], mats=['air', 'glass', 'soft', 'hard', 'air'], 
               ns=[lambda x:n_air, lambda x:n_glass, lambda x:n_soft, lambda x:n_hard,lambda x:n_air],
              maxhs=[2, .02, 2, .1, 2], wl=1.8e-6, scale=B.scale, curve=80)

In [ ]:
Draw(CF([n for n in A.ns]), A.mesh)

In [ ]:
k_low, k_high = A.k0 * np.min(A.ns) * A.scale, A.k0 * np.max(A.ns) * A.scale

outer = 'h2'
Ktype = 'kappa'
nu = 1

In [ ]:
%matplotlib notebook
plotlogf(A.determinant, .9995*k_low, 1.00001*k_low, -.01,.01, nu, outer,
         iref=100, rref=100, levels=100, figsize=(12,8))

In [ ]:
guess = np.array(.9999*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer, Ktype), tol = 1e-16)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer, Ktype)))


# Bragg Exact Fields

In [ ]:
Fs = A.all_fields(beta1, nu=nu, outer=outer, Ktype=Ktype)

In [ ]:
Draw(Fs['Ez'], A.mesh)

In [ ]:
Draw(Fs['Hz'], A.mesh)

In [ ]:
Draw(Fs['Etv'].real, A.mesh, vectors={'grid_size':1500})


# Looks possible to do wavelength loss study here for varying extinction coefficients in the polymer.

Hopefully not all the effect disappears at realistic values